<a href="https://colab.research.google.com/github/HeatherDriver/MathGraph/blob/main/03_NER_Case_Tag_Dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install intervaltree

  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=c46886948142b18b3b388ac16f65081f7f57e23defd45d48565d1e4d0196d54a
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from google.colab import drive, userdata
import pickle
import random
import re
from collections import defaultdict
from intervaltree import Interval, IntervalTree
from collections import Counter
import json

In [ ]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [ ]:
# Define nested_dict function
def nested_dict():
  return defaultdict(nested_dict)

# Define file read function
def read_pickle(dict_file):
  with open(dict_file, 'rb') as file:
    return pickle.load(file)

# Split CamelCase to s e p a r a t e d text
def split_camel_case(text):
    # Use regex to insert a space before uppercase letters preceded by lowercase letters
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

In [ ]:
# Existing tags from DistilBERT
tag_checkpoint = "dslim/distilbert-NER"
tag_tokenizer = AutoTokenizer.from_pretrained(tag_checkpoint, do_lower_case=False)
tag_model = AutoModelForTokenClassification.from_pretrained(tag_checkpoint)

tags = pipeline("ner", model=tag_model, tokenizer=tag_tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
tag_tokenizer.is_fast

True

In [ ]:
# Methods built from DistilBERT NER

def idx2string(text):
    # Split on whitespace or punctuation, punctuation is used for tokens
    words = re.findall(r'\b\w+\b|[^\s\w]', text)
    my_dict = {i: word for i, word in enumerate(words)}
    return my_dict

def tokenidx2words(ner_results):
  my_dict = dict()
  for sub_dict in ner_results:
    word, index = sub_dict['word'], sub_dict['index']
    my_dict[index] = word
  return my_dict

def tokenidx2entity(ner_results):
  my_dict = dict()
  for sub_dict in ner_results:
    entity, index = sub_dict['entity'], sub_dict['index']
    my_dict[index] = entity
  return my_dict

def idx2wordpos(text, idx2string):
  my_dict = {}
  current_pos = 0
  for idx, word in idx2string.items():
    start = text.index(word, current_pos)
    end = start + len(word)
    my_dict[idx] = (start, end)
    current_pos = end
  return my_dict

def tokenidx2entityword(example2wordidx, ner_results):
  # Create an IntervalTree
  tree = IntervalTree(Interval(start, end, key) for key, (start, end) in example2wordidx.items())

  my_dict = dict()
  for sub_dict in ner_results:
    tag_start, tag_end = sub_dict['start'], sub_dict['end']
    tag_entity = sub_dict['entity']
    token_idx = sub_dict['index']

    overlapping_intervals = tree[tag_start : tag_end] # Intervals which contain the entitry recognised
    for interval in overlapping_intervals:
      my_dict[token_idx] = {'entity' : tag_entity, 'word_idx': interval.data}

  return my_dict

In [ ]:
# Get naming convention from scraped data
topics = read_pickle("topics.pkl")

print(f"topic_dict dictionary length: {len(topics)}")
random_sample = random.sample(list(topics.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n{value}")

topic_dict dictionary length: 10
Applied Mathematics:
['Business', 'Complex Systems', 'Control Theory', 'Data Visualization', 'Dynamical Systems', 'Engineering', 'Ergodic Theory', 'Game Theory', 'Information Theory', 'Inverse Problems', 'Numerical Methods', 'Optimization', 'Population Dynamics', 'Signal Processing']
Calculus and Analysis:
['Calculus', 'Calculus of Variations', 'Catastrophe Theory', 'Complex Analysis', 'Differential Equations', 'Differential Forms', 'Differential Geometry', 'Dynamical Systems', 'Fixed Points', 'Functional Analysis', 'Functions', 'General Analysis', 'Generalized Functions', 'Harmonic Analysis', 'Inequalities', 'Integral Transforms', 'Inversion Formulas', 'Manifolds', 'Measure Theory', 'Metrics', 'Norms', 'Operator Theory', 'Polynomials', 'Roots', 'Series', 'Singularities', 'Special Functions']
Topology:
['Algebraic Topology', 'Bundles', 'Cohomology', 'General Topology', 'Knot Theory', 'Low-Dimensional Topology', 'Manifolds', 'Point-Set Topology', 'Spaces

In [ ]:
# Get naming convention from scraped data
sub_topics = read_pickle("sub_topics.pkl")

my_dict = dict()
for key, value in sub_topics.items():
  split_key = split_camel_case(key)
  my_dict[split_key] = value

sub_topics = my_dict

print(f"sub_topics dictionary length: {len(sub_topics)}")

for key, value in random_sample:
  print(f"{key}:\n{sub_topics[key]}")

sub_topics dictionary length: 10
Applied Mathematics:
['Accounting', 'Actuarial Mathematics', 'Economics', 'Finance', 'Cellular Automata', 'Chaos', 'Fractals', 'Control Theory', 'Filter', 'Jordan Canonical Form', 'Kalman Filter', "Little's Law", 'Pontryagin Maximum Principle', 'Riccati Differential Equation', 'Schur Decomposition', 'Similarity Transformation', 'Transfer Function', 'Anti-Aliasing', 'Antialiasing', 'Bar Chart', 'Binary Plot', 'Cobweb Plot', 'Contour Line', 'Contour Plot', 'Data Cube', 'de Finetti Diagram', 'Equipotential Curve', 'Function Graph', 'Inside-Outside Plot', 'Level Curve', 'Level Set', 'Level Surface', 'Moiré Pattern', 'Ordinate', 'Phase Curve', 'Phase Flow', 'Phase Plane', 'Phase Portrait', 'Phase Space', 'Pie Chart', 'Poincaré Section', 'Pólya Plot', 'Recurrence Plot', 'Saunders Graphic', 'Scatter Diagram', 'Scatter Plot', 'Scatterplot', 'Simplex Plot', 'Slope Field', 'Sparkline', "Strang's Strange Figures", 'Surface of Section', 'Ternary Diagram', 'Ternary 

In [ ]:
# Merge Topics and Subtopics to one dictionary
for key, value in topics.items():
  if key in sub_topics and sub_topics[key] is not None:
    value.extend(sub_topics[key])
    # Convert to set and back to list to remove duplicates and sort
    new_value = list(set(value))
    new_value.sort()
    topics[key] = new_value

In [ ]:
print(f"topics dictionary length: {len(topics)}")

for key, value in random_sample:
  print(f"{key}:\n{topics[key]}")

topics dictionary length: 10
Applied Mathematics:
['2x mod 1 Map', 'Accounting', 'Accumulation Point', 'Action', 'Actuarial Mathematics', 'Adiabatic Invariant', 'All-to-All Communication', 'Allometric', 'Ambrose-Kakutani Theorem', 'Angel Problem', 'Annealing', 'Anonymous', 'Anosov Automorphism', 'Anosov Diffeomorphism', 'Anosov Flow', 'Anosov Map', 'Ant Colony Algorithm', 'Anti-Aliasing', 'Antialiasing', 'Approximation Theory', "Arnold's Cat Map", "Arrow's Impossibility Theorem", "Arrow's Paradox", 'Asymptotic Equipartition Property', 'Auction', 'Average Power', 'Axiom A Flow', "Baker's Map", 'Banzhaf Power Index', 'Bar Chart', 'Bifurcation', 'Bifurcation Theory', 'Biggest Little Hexagon', 'Biggest Little Polygon', 'Binary Plot', 'Biotic Potential', "Birkhoff's Ergodic Theorem", "Birkhoff's Inequality", 'Borel Determinacy Theorem', 'Branch and Bound Algorithm', 'Broadcasting', 'Building Problem', 'Business', 'Byzantine Generals Problem', 'Cake Cutting', 'Carrying Capacity', 'Cascade', 

In [ ]:
alg_2 = read_pickle("alg_2.pkl")
alg_3 = read_pickle("alg_3.pkl")

# merge alg_2 and alg_3 into 1 dictionary
alg_dict = dict()

for key, value in alg_2.items():
  alg_dict[key] = value

for key, value in alg_3.items():
  alg_dict[key] = value

print(f"alg_dict dictionary length: {len(alg_dict)}")
random_sample = random.sample(list(alg_dict.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n{value}")

alg_dict dictionary length: 18
Matrix Norms:
['Compatible', 'Euclidean Norm', 'Frobenius Norm', 'Hilbert-Schmidt Norm', 'Matrix Norm', 'Matrix p-Norm', 'Maximum Absolute Column Sum Norm', 'Maximum Absolute Row Sum Norm', 'Natural Norm', 'Spectral Norm']
Matrix Groups:
['General Linear Group', 'Heisenberg Group', 'Lie-Type Group', 'Linear Algebraic Group', "Maschke's Theorem", 'Matrix Group', 'Orthogonal Group', 'Rotation Group', 'Special Linear Group', 'Special Orthogonal Group', 'Special Unitary Group', 'Symplectic Group']
General Linear Algebra:
['Alternating Multilinear Form', 'Bilinear Basis', 'Complex Vector Space', "Fredholm's Theorem", 'Fundamental Matrix Subspaces', 'Fundamental Theorem of Linear Algebra', 'Haar Condition', 'Hermitian Inner Product', 'Invertible Linear Map', 'Kernel', 'Linear Algebra', 'Linear Combination', 'Linear Function', 'Linear Transformation Kernel', 'Linearly Independent', 'Lorentzian Inner Product', 'Null Space', 'Nullity', 'Nullspace', 'Orthogonal Com

In [ ]:
all_topics = dict()
for key, value in alg_dict.items():
  all_topics[key] = value

for key, value in topics.items():
  all_topics[key] = value

all_topics['Linear Algebra'] = all_topics['General Linear Algebra']
del all_topics['General Linear Algebra']

print(f"all_topics dictionary length: {len(all_topics)}")
random_sample = random.sample(list(all_topics.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n{value}")

all_topics dictionary length: 28
Linear Independence:
["Fredholm's Theorem", 'Linearly Dependent Functions', 'Linearly Dependent Vectors', 'Linearly Independent', 'Orthogonal', 'Vector Basis', 'Wronskian']
Topology:
['0-Connected', '1-Connected', 'Absolute Retract', 'Abstract Manifold', 'Abstract Simplicial Complex', 'Abstract Topological Space', 'Abstract Vector Space', 'Acyclic Chain Complex', "Adams' Theorem", 'Affine Hull', 'Ahlfors-Bers Theorem', 'Aleksandrov-Čech Cohomology', "Alexander's Horned Sphere", 'Alexander-Spanier Cohomology', 'Algebraic Gadget', 'Algebraic K-Theory', 'Algebraic Manifold', 'Algebraic Topology', 'Almost Everywhere Convergence', 'Alternating Knots', 'Ambient Isotopy', 'Amphichiral Knots', 'Analytic Torsion', 'Anchor', 'Anosov Flow', 'Antipodal Map', "Antoine's Horned Sphere", "Antoine's Necklace", 'Approximation Problem', 'Approximation Property', 'Arc Component', 'Arcwise-Connected', 'Associated Fiber Bundle', 'Associated Vector Bundle', 'Atiyah-Singer In

In [ ]:
# Check 1: 1 mapping of keys values
my_dict = dict()
for key, value_list in all_topics.items():
  for value in value_list:
    if value not in my_dict.keys():
      my_dict[value] = [key]
    else:
      my_dict[value].extend([key])

map_to = dict()
for key, value_list in my_dict.items():
  if len(value_list) == 2:
    if 'Recreational Mathematics' in value_list:
      value_list.remove('Recreational Mathematics')
      map_to[key] = value_list[0]

# Create a list of keys to delete to avoid changing dictionary size during iteration
keys_to_delete = [key for key in my_dict if key in map_to]

# Delete the keys
for key in keys_to_delete:
  del my_dict[key]
  # Delete from all_topics
  all_topics['Recreational Mathematics'].remove(key)

In [ ]:
# Choose linear algebra topic classification over other types
all_topics_keys_to_remove = dict()
keys_to_delete = list()

for key, value_list in my_dict.items():
  if len(value_list) == 2:
    if 'Determinants' in value_list:
      value_list.remove('Determinants')
      all_topics_keys_to_remove[value_list[0]] = key
      keys_to_delete.append(key)

# Delete the keys
for key in keys_to_delete:
  del my_dict[key]

for key, value in all_topics_keys_to_remove.items():
  # Delete from all_topics
  all_topics[key].remove(value)

In [ ]:
my_dict = dict()
for key, value_list in all_topics.items():
  for value in value_list:
    if value not in my_dict.keys():
      my_dict[value] = [key]
    else:
      my_dict[value].extend([key])

In [ ]:
to_remove = {
    "Fundamental Matrix Subspaces": ['Matrix Decomposition', 'Matrix Operations'],
    "Matrix Diagonalization": ['Matrix Decomposition', 'Matrix Eigenvalues'],
    "Stochastic Matrix": ['Algebra', 'Probability and Statistics'],
    "Hadamard Matrix": ['Algebra', 'Foundations of Mathematics', 'Recreational Mathematics', 'Discrete Mathematics'],
    "Polynomial Matrix": [ 'Algebra', 'Calculus and Analysis'],
    "Sylvester Matrix": ['Algebra', 'Calculus and Analysis'],
    "Matrix Polynomial": ['Algebra', 'Calculus and Analysis']
    }

for key, value_list in to_remove.items():
  for val in value_list:
    all_topics[val].remove(key)

my_dict = dict()
for key, value_list in all_topics.items():
  for value in value_list:
    if value not in my_dict.keys():
      my_dict[value] = [key]
    else:
      my_dict[value].extend([key])

In [ ]:
to_remove = {
    "Lie Algebra": ['Lie Theory', 'Lie Algebra']
    ,"Fredholm's Theorem": ['Linear Independence', 'Matrix Properties']
    ,"Jacobi Identities": ['Lie Algebra', 'Lie Groups']
    ,"Flag Manifold": ['Lie Groups', 'Calculus and Analysis']
    ,"Lie-Type Group": ['Lie Groups', 'Discrete Mathematics']
    ,"Special Linear Group": ['Lie Groups', 'Discrete Mathematics']
    ,"Fundamental Theorem of Linear Algebra": ['Matrix Decomposition', 'Matrix Operations']
    ,"Schur Decomposition": ['Matrix Eigenvalues', 'Applied Mathematics']
    ,"Conjugate Transpose": ['Matrix Types', 'Calculus and Analysis']
    ,"Analytic Torsion": ['Calculus and Analysis', 'Topology']
    ,"Byzantine Generals Problem": ['Algebra', 'Applied Mathematics']
    ,"Code": ['Algebra', 'Foundations of Mathematics']
    ,"Commutative Diagram": ['Algebra', 'Foundations of Mathematics']
    ,"Contractible": ['Foundations of Mathematics', 'Topology']
    ,"Cyclotomic Polynomial": ['Foundations of Mathematics', 'Discrete Mathematics']
    ,"Elliptic Curves": ['Algebra', 'Geometry']
    ,"Encoding": ['Algebra', 'Foundations of Mathematics']
    ,"Endomorphism": ['Algebra', 'Foundations of Mathematics', 'Applied Mathematics', 'Geometry']
    ,"Ergodic Theory": ['Algebra', 'Applied Mathematics', 'Geometry', 'Calculus and Analysis']
    ,"Gerbe": [ 'Foundations of Mathematics', 'Topology']
    ,"Gröbner Basis": ['Algebra', 'Discrete Mathematics']
    ,"Icosahedral Equation": ['Algebra', 'Calculus and Analysis']
    ,"Kac Formula": ['Algebra', 'Probability and Statistics']
    ,"Minimax Polynomial": ['Probability and Statistics', 'Calculus and Analysis']
    ,"Modularity Theorem": ['Algebra', 'Number Theory']
    ,"Noise": ['Algebra', 'Applied Mathematics']
    ,"Normal Polynomial": ['Calculus and Analysis', 'Number Theory']
    ,"Octahedral Equation": ['Geometry', 'Calculus and Analysis']
    ,"Polynomial Sequence": ['Algebra', 'Calculus and Analysis']
    ,"Power Polynomial": ['Calculus and Analysis', 'Discrete Mathematics']
    ,"Real Vector Space": ['Algebra', 'Topology']
    ,"Reidemeister Torsion": ['Calculus and Analysis', 'Topology']
    ,"Taniyama-Shimura Conjecture": ['Algebra', 'Topology']
    ,"Tetrahedral Equation": ['Algebra', 'Geometry']
    ,"Vector Field": ['Applied Mathematics', 'Topology']
    ,"Whitehead Torsion": ['Algebra', 'Calculus and Analysis']
    ,"Zero Map": ['Algebra', 'Foundations of Mathematics', 'Calculus and Analysis']
    ,"Borel Hierarchy": ['Foundations of Mathematics', 'Topology']
    ,"Busy Beaver": ['Foundations of Mathematics', 'Recreational Mathematics']
    ,"Circle Packing": ['Foundations of Mathematics', 'Recreational Mathematics', 'Discrete Mathematics']
    ,"Closed Map": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Collatz Problem": ['Foundations of Mathematics', 'Recreational Mathematics']
    ,"Complete Metric Space": ['Foundations of Mathematics', 'Topology']
    ,"Contingent Cone": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Dense": ['Foundations of Mathematics', 'Topology']
    ,"Domain": ['Foundations of Mathematics', 'Calculus and Analysis', 'Topology']
    ,"Endpoint": ['Foundations of Mathematics', 'Geometry']
    ,"F_sigma Set": ['Foundations of Mathematics', 'Topology']
    ,"First Category": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"G_delta Set": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Generalized Reeb Component": ['Foundations of Mathematics', 'Topology']
    ,"Helly's Theorem": ['Foundations of Mathematics', 'Geometry']
    ,"Hofstadter's Q-Sequence": ['Foundations of Mathematics', 'Discrete Mathematics']
    ,"Hofstadter-Conway $10,000 Sequence": ['Foundations of Mathematics', 'Discrete Mathematics']
    ,"Injection": ['Foundations of Mathematics', 'Topology']
    ,"Lebesgue Covering Dimension": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Logistic Map": ['Foundations of Mathematics', 'Applied Mathematics', 'Discrete Mathematics']
    ,"Logistic Map--r=2": ['Foundations of Mathematics', 'Applied Mathematics']
    ,"Logistic Map--r=4": ['Foundations of Mathematics', 'Applied Mathematics']
    ,"Metric Topology": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Metrizable Topology": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Net": ['Foundations of Mathematics', 'Probability and Statistics']
    ,"Nice Vector Field": ['Foundations of Mathematics', 'Topology']
    ,"Pseudoconcave Function": ['Foundations of Mathematics', 'Topology']
    ,"Pseudoconvex Function": ['Foundations of Mathematics', 'Topology']
    ,"Quasi-Concave Function": ['Foundations of Mathematics', 'Topology']
    ,"Quasi-Convex Function": ['Foundations of Mathematics', 'Topology']
    ,"Reeb Component": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Reebless": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Stochastic": ['Foundations of Mathematics', 'Applied Mathematics', 'Geometry', 'Number Theory']
    ,"Surjection": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Sutured Manifold": ['Foundations of Mathematics', 'Topology']
    ,"Taut Foliation": ['Foundations of Mathematics', 'Topology']
    ,"Transversely Orientable Foliation": ['Foundations of Mathematics', 'Topology']
    ,"Uniform Convexity": ['Foundations of Mathematics', 'Topology']
    ,"Zero Set": ['Foundations of Mathematics', 'Topology']
    ,"Lévy Flight": ['Probability and Statistics', 'Applied Mathematics', 'Geometry']
    ,"Lévy Walk": ['Probability and Statistics', 'Applied Mathematics', 'Geometry']
    ,"Quantum Stochastic Calculus": ['Probability and Statistics', 'Geometry', 'Number Theory']
    ,"Sampling": ['Applied Mathematics', 'Discrete Mathematics']
    ,"Seed": ['Probability and Statistics', 'Calculus and Analysis']
    ,"2x mod 1 Map": ['Calculus and Analysis', 'Discrete Mathematics']
    ,"Anosov Flow": ['Calculus and Analysis', 'Topology']
    ,"Axiom A Flow": ['Calculus and Analysis', 'Topology']
    ,"Cake Cutting": ['Geometry', 'Discrete Mathematics']
    ,"Dye's Theorem": ['Applied Mathematics', 'Geometry']
    ,"Ergodic Measure": ['Applied Mathematics', 'Geometry']
    ,"Geodesic Flow": ['Calculus and Analysis', 'Topology']
    ,"KMS Condition": ['Applied Mathematics', 'Geometry', 'Calculus and Analysis']
    ,"Kronecker-Weyl Theorem": ['Applied Mathematics', 'Geometry']
    ,"Kubo-Martin-Schwinger Condition": ['Geometry', 'Calculus and Analysis', 'Number Theory']
    ,"Lévy Process": ['Geometry', 'Number Theory']
    ,"Number Partitioning Problem": ['Applied Mathematics', 'Geometry']
    ,"Ornstein's Theorem": ['Geometry', 'Number Theory']
    ,"Pesin Theory": ['Geometry', 'Number Theory']
    ,"Phase Space": ['Applied Mathematics', 'Topology']
    ,"Poisson Process": ['Geometry', 'Number Theory']
    ,"Smale Horseshoe Map": ['Applied Mathematics', 'Calculus and Analysis']
    ,"Connex": ['Geometry', 'Calculus and Analysis']
    ,"Dehn Invariant": ['Geometry', 'Recreational Mathematics']
    ,"Integer Triangle": ['Geometry', 'Recreational Mathematics']
    ,"Kobon Triangle": ['Geometry', 'Recreational Mathematics']
    ,"Pizza Theorem": ['Geometry', 'Recreational Mathematics']
    ,"Subset Sum Problem": ['Geometry', 'Discrete Mathematics']
    ,"Banach Space": ['Recreational Mathematics', 'Topology']
    ,"Hilbert Space": ['Recreational Mathematics', 'Topology']
    ,"Perrin Sequence": ['Recreational Mathematics', 'Discrete Mathematics']
    ,"Pi": ['Recreational Mathematics', 'Discrete Mathematics']
}

for key, value_list in to_remove.items():
  for val in value_list:
    all_topics[val].remove(key)

my_dict = dict()
for key, value_list in all_topics.items():
  for value in value_list:
    if value not in my_dict.keys():
      my_dict[value] = [key]
    else:
      my_dict[value].extend([key])

# Final 1:1 check
my_dict = dict()
for key, value_list in my_dict.items():
  if len(value_list) != 1:
    print(f"{key}: {value_list}")

In [ ]:
def capitalise_key(key):
  words = key.split()
  if len(words) == 1:
    to_return = words[0].capitalize()
  else:
    first = words[0].capitalize()
    rest = words[1:]
    to_return = ' '.join(rest).lower()
    to_return = f"{first} {to_return}"
  return to_return

In [ ]:
def generate_ner_tags(classification_dict):
  # Generates hierarchical tags based on the key
  transformations = {
        "lower": str.lower,
        "upper": str.upper,
        "title": str.title,
        "capitalise": capitalise_key,
  }
  my_dict = dict()
  for key, value_list in classification_dict.items():
    key = key.replace(" ", "_").upper()
    for value in value_list:
      transformed_values = {}
      for name, transform_fn in transformations.items():
        transformed_values[name] = transform_fn(value)

      for value in transformed_values.values():
        text_index = idx2string(value)
        my_dict[value] = {
          "text_index": text_index,
          "tokens" : tag_tokenizer(value).tokens(),
          "input_ids" : tag_tokenizer.encode(value),
          }
        my_list = []
        for k, v in my_dict[value]['text_index'].items():
          for token, input_id in zip(my_dict[value]['tokens'], my_dict[value]['input_ids']):
            if v.startswith(token):
              my_list.append(input_id)
          my_dict[value]['ner_mapper'] = my_list
        for i, entry in enumerate(my_list):
          if i == 0:
            my_dict[value]['ner_tags'] = [f'B_{key}']
          else:
            my_dict[value]['ner_tags'].append(f'I_{key}')
  return my_dict

In [ ]:
def generate_ner_tags_from_keys(classification_dict, main_name):
  # Generates tags for keys based on the name or text supplied
  main_name = main_name.upper()
  transformations = {
      "lower": str.lower,
        "upper": str.upper,
        "title": str.title,
        "capitalise": capitalise_key,

  }
  my_dict = {}
  for key in classification_dict.keys():
    transformed_values = {name: transform_fn(key) for name, transform_fn in transformations.items()}
    for value in transformed_values.values():
      text_index = idx2string(value)
      tokens = tag_tokenizer(value).tokens()
      input_ids = tag_tokenizer.encode(value)

      ner_mapper = []
      for k, v in text_index.items():
        for token, input_id in zip(tokens, input_ids):
          if v.startswith(token):
            ner_mapper.append(input_id)

      # Generate NER tags based on matched tokens
      ner_tags = []
      for i, token in enumerate(tokens):
        if i == 0:  # First token gets the 'B_' tag
          ner_tags.append(f'B_{main_name}')
        else:  # Other tokens get 'I_' tag
          ner_tags.append(f'I_{main_name}')

      # Save the results in the dictionary
      my_dict[value] = {
                "text_index": text_index,
                "tokens": tokens,
                "input_ids": input_ids,
                "ner_mapper": ner_mapper,
                "ner_tags": ner_tags,
            }

    return my_dict

In [ ]:
def generate_ner_tags_from_word(classification_list, main_name):
  # Generates tags for keys based on the name or text supplied
  main_name = main_name.upper()
  transformations = {
      "lower": str.lower,
        "upper": str.upper,
        "title": str.title,
        "capitalise": capitalise_key,

  }
  my_dict = {}
  for key in classification_list:
    transformed_values = {name: transform_fn(key) for name, transform_fn in transformations.items()}
    for value in transformed_values.values():
      text_index = idx2string(value)
      tokens = tag_tokenizer(value).tokens()
      input_ids = tag_tokenizer.encode(value)

      ner_mapper = []
      for k, v in text_index.items():
        for token, input_id in zip(tokens, input_ids):
          if v.startswith(token):
            ner_mapper.append(input_id)

      # Generate NER tags based on matched tokens
      ner_tags = []
      for i, token in enumerate(tokens):
        if i == 0:  # First token gets the 'B_' tag
          ner_tags.append(f'B_{main_name}')
        else:  # Other tokens get 'I_' tag
          ner_tags.append(f'I_{main_name}')

      # Save the results in the dictionary
      my_dict[value] = {
                "text_index": text_index,
                "tokens": tokens,
                "input_ids": input_ids,
                "ner_mapper": ner_mapper,
                "ner_tags": ner_tags,
            }

    return my_dict

In [ ]:
# Generate the NER tags per dictionary
_all_classifications = generate_ner_tags(all_topics)

In [ ]:
print(f"topic_dict_ner dictionary length: {len(_all_classifications)}")
random_sample = random.sample(list(_all_classifications.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n{value}")

topic_dict_ner dictionary length: 21696
zero element:
{'text_index': {0: 'zero', 1: 'element'}, 'tokens': ['[CLS]', 'zero', 'element', '[SEP]'], 'input_ids': [101, 6756, 5290, 102], 'ner_mapper': [6756, 5290], 'ner_tags': ['B_ALGEBRA', 'I_ALGEBRA']}
POSTAGE STAMP PROBLEM:
{'text_index': {0: 'POSTAGE', 1: 'STAMP', 2: 'PROBLEM'}, 'tokens': ['[CLS]', 'P', '##OS', '##TA', '##GE', 'ST', '##AM', '##P', 'PR', '##O', '##BL', '##EM', '[SEP]'], 'input_ids': [101, 153, 9025, 9159, 16523, 23676, 10964, 2101, 11629, 2346, 13360, 15577, 102], 'ner_mapper': [153, 23676, 153, 11629], 'ner_tags': ['B_NUMBER_THEORY', 'I_NUMBER_THEORY', 'I_NUMBER_THEORY', 'I_NUMBER_THEORY']}
Chinese hypothesis:
{'text_index': {0: 'Chinese', 1: 'hypothesis'}, 'tokens': ['[CLS]', 'Chinese', 'hypothesis', '[SEP]'], 'input_ids': [101, 1922, 11066, 102], 'ner_mapper': [1922, 11066], 'ner_tags': ['B_NUMBER_THEORY', 'I_NUMBER_THEORY']}


In [ ]:
all_classifications = dict()

for key, sub_dict in _all_classifications.items():
  ner = list('O' * len(sub_dict['input_ids']))
  for match_input_id, ner_tag in zip(sub_dict['ner_mapper'], sub_dict['ner_tags']):
    idx = sub_dict['input_ids'].index(match_input_id)
    ner[idx] = ner_tag
    ner = [i.replace('_', '-') for i in ner]
    all_classifications[key] = {
        'text_index': sub_dict['text_index'],
        'tokens': sub_dict['tokens'],
        'input_ids': sub_dict['input_ids'],
        'ner_mapper': sub_dict['ner_mapper'],
        'ner_tags': ner,
    }

In [ ]:
print(f"topic_dict_ner dictionary length: {len(all_classifications)}")

for key, value in random_sample:
  print(f"{key}:\n{all_classifications[key]}")

topic_dict_ner dictionary length: 21696
zero element:
{'text_index': {0: 'zero', 1: 'element'}, 'tokens': ['[CLS]', 'zero', 'element', '[SEP]'], 'input_ids': [101, 6756, 5290, 102], 'ner_mapper': [6756, 5290], 'ner_tags': ['O', 'B-ALGEBRA', 'I-ALGEBRA', 'O']}
POSTAGE STAMP PROBLEM:
{'text_index': {0: 'POSTAGE', 1: 'STAMP', 2: 'PROBLEM'}, 'tokens': ['[CLS]', 'P', '##OS', '##TA', '##GE', 'ST', '##AM', '##P', 'PR', '##O', '##BL', '##EM', '[SEP]'], 'input_ids': [101, 153, 9025, 9159, 16523, 23676, 10964, 2101, 11629, 2346, 13360, 15577, 102], 'ner_mapper': [153, 23676, 153, 11629], 'ner_tags': ['O', 'I-NUMBER-THEORY', 'O', 'O', 'O', 'I-NUMBER-THEORY', 'O', 'O', 'I-NUMBER-THEORY', 'O', 'O', 'O', 'O']}
Chinese hypothesis:
{'text_index': {0: 'Chinese', 1: 'hypothesis'}, 'tokens': ['[CLS]', 'Chinese', 'hypothesis', '[SEP]'], 'input_ids': [101, 1922, 11066, 102], 'ner_mapper': [1922, 11066], 'ner_tags': ['O', 'B-NUMBER-THEORY', 'I-NUMBER-THEORY', 'O']}


In [ ]:
file_name = 'all_classifications.pkl'

with open(file_name, 'wb') as file:
  pickle.dump(all_classifications, file)